In [ ]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/datasets/pjsingh/liveliness")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aaryan30
Your Kaggle Key: ··········


100%|██████████| 141M/141M [00:02<00:00, 69.6MB/s]


In [ ]:
# -------------------------------------- profile_detection ---------------------------------------
detect_frontal_face = 'profile_detection/haarcascades/haarcascade_frontalface_alt.xml'
detect_perfil_face = 'profile_detection/haarcascades/haarcascade_profileface.xml'

# Parametros del modelo, la imagen se debe convertir a una de tamaño 48x48 en escala de grises
w,h = 48,48
rgb = False
# definir la relacion de aspecto del ojo EAT
# definir el numero de frames consecutivos que debe estar por debajo del umbral
EYE_AR_THRESH = 0.23 #baseline
EYE_AR_CONSEC_FRAMES = 1

# eye landmarks
eye_landmarks = "blink_detection/model_landmarks/shape_predictor_68_face_landmarks.dat"

In [ ]:
import tensorflow as tf
from easydict import EasyDict as edict

import ops


def get_vgg16_conv5(input, params):
    layers = edict()

    layers.conv1_1 = ops.conv2D(input=input, shape=(3, 3, 64), name='conv1_1', params=params)
    layers.conv1_1_relu = ops.activate(input=layers.conv1_1, name='conv1_1_relu', act_type='relu')
    layers.conv1_2 = ops.conv2D(input=layers.conv1_1_relu, shape=(3, 3, 64), name='conv1_2', params=params)
    layers.conv1_2_relu = ops.activate(input=layers.conv1_2, name='conv1_2_relu', act_type='relu')
    layers.pool1 = ops.max_pool(input=layers.conv1_2_relu, name='pool1')

    layers.conv2_1 = ops.conv2D(input=layers.pool1, shape=(3, 3, 128), name='conv2_1', params=params)
    layers.conv2_1_relu = ops.activate(input=layers.conv2_1, name='conv2_1_relu', act_type='relu')
    layers.conv2_2 = ops.conv2D(input=layers.conv2_1_relu, shape=(3, 3, 128), name='conv2_2', params=params)
    layers.conv2_2_relu = ops.activate(input=layers.conv2_2, name='conv2_2_relu', act_type='relu')
    layers.pool2 = ops.max_pool(input=layers.conv2_2_relu, name='pool2')

    layers.conv3_1 = ops.conv2D(input=layers.pool2, shape=(3, 3, 256), name='conv3_1', params=params)
    layers.conv3_1_relu = ops.activate(input=layers.conv3_1, name='conv3_1_relu', act_type='relu')
    layers.conv3_2 = ops.conv2D(input=layers.conv3_1_relu, shape=(3, 3, 256), name='conv3_2', params=params)
    layers.conv3_2_relu = ops.activate(input=layers.conv3_2, name='conv3_2_relu', act_type='relu')
    layers.conv3_3 = ops.conv2D(input=layers.conv3_2_relu, shape=(3, 3, 256), name='conv3_3', params=params)
    layers.conv3_3_relu = ops.activate(input=layers.conv3_3, name='conv3_3_relu', act_type='relu')
    layers.pool3 = ops.max_pool(input=layers.conv3_3_relu, name='pool3')

    layers.conv4_1 = ops.conv2D(input=layers.pool3, shape=(3, 3, 512), name='conv4_1', params=params)
    layers.conv4_1_relu = ops.activate(input=layers.conv4_1, name='conv4_1_relu', act_type='relu')
    layers.conv4_2 = ops.conv2D(input=layers.conv4_1_relu, shape=(3, 3, 512), name='conv4_2', params=params)
    layers.conv4_2_relu = ops.activate(input=layers.conv4_2, name='conv4_2_relu', act_type='relu')
    layers.conv4_3 = ops.conv2D(input=layers.conv4_2_relu, shape=(3, 3, 512), name='conv4_3', params=params)
    layers.conv4_3_relu = ops.activate(input=layers.conv4_3, name='conv4_3_relu', act_type='relu')
    layers.pool4 = ops.max_pool(input=layers.conv4_3_relu, name='pool4')

    layers.conv5_1 = ops.conv2D(input=layers.pool4, shape=(3, 3, 512), name='conv5_1', params=params)
    layers.conv5_1_relu = ops.activate(input=layers.conv5_1, name='conv5_1_relu', act_type='relu')
    layers.conv5_2 = ops.conv2D(input=layers.conv5_1_relu, shape=(3, 3, 512), name='conv5_2', params=params)
    layers.conv5_2_relu = ops.activate(input=layers.conv5_2, name='conv5_2_relu', act_type='relu')
    layers.conv5_3 = ops.conv2D(input=layers.conv5_2_relu, shape=(3, 3, 512), name='conv5_3', params=params)
    layers.conv5_3_relu = ops.activate(input=layers.conv5_3, name='conv5_3_relu', act_type='relu')

    return layers

def get_vgg16_pool5(input, params):
    layers = get_vgg16_conv5(input, params)
    layers.pool5 = ops.max_pool(input=layers.conv5_3_relu, name='pool5')

    return layers

def get_prob(input, params, num_class=1000, is_train=True):
    # Get pool5
    layers = get_vgg16_pool5(input, params)
    layers.fc6 = ops.fully_connected(input=layers.pool5, num_neuron=4096, name='fc6', params=params)
    if is_train:
        layers.fc6 = tf.nn.dropout(layers.fc6, keep_prob=0.5)
    layers.fc6_relu = ops.activate(input=layers.fc6, act_type='relu', name='fc6_relu')
    layers.fc7 = ops.fully_connected(input=layers.fc6_relu, num_neuron=4096, name='fc7', params=params)
    if is_train:
        layers.fc7 = tf.nn.dropout(layers.fc7, keep_prob=0.5)
    layers.fc7_relu = ops.activate(input=layers.fc7, act_type='relu', name='fc7_relu')
    layers.fc8 = ops.fully_connected(input=layers.fc7_relu, num_neuron=num_class, name='fc8', params=params)
    layers.prob = tf.nn.softmax(layers.fc8)
    return layers

In [ ]:
import numpy as np
import cv2, os
import matplotlib
import sys
import warnings
import dlib
matplotlib.use('Agg')
from tqdm import tqdm
sys.path.append(os.path.dirname(os.path.dirname(__file__)))
import pickle
from py_utils.face_utils import lib
from py_utils.vid_utils import proc_vid as pv
from py_utils.plot_utils import plot


class Solu(object):

    def __init__(self,
                 input_vid_path,
                 output_height=300,
                 ):
        # Input video
        self.input_vid_path = input_vid_path
        # parse video
        print('Parsing video {}'.format(str(self.input_vid_path)))
        self.imgs, self.frame_num, self.fps, self.img_w, self.img_h = pv.parse_vid(str(self.input_vid_path))
        if len(self.imgs) != self.frame_num:
            warnings.warn('Frame number is not consistent with the number of images in video...')
            self.frame_num = len(self.imgs)
        print('Eye blinking solution is building...')

        self._set_up_dlib()

        self.output_height = output_height
        factor = float(self.output_height) / self.img_h

        # Resize imgs for final video generation
        # Resize self.imgs according to self.output_height
        self.aligned_imgs = []
        self.left_eyes = []
        self.right_eyes = []

        self.resized_imgs = []
        print('face aligning...')
        for i, im in enumerate(tqdm(self.imgs)):
            face_cache = lib.align(im[:, :, (2,1,0)], self.front_face_detector, self.lmark_predictor)
            if len(face_cache) == 0:
                self.left_eyes.append(None)
                self.right_eyes.append(None)
                continue

            if len(face_cache) > 1:
                raise ValueError('{} faces are in image, we only support one face in image.')

            aligned_img, aligned_shapes_cur = lib.get_aligned_face_and_landmarks(im, face_cache)
            # crop eyes
            leye, reye = lib.crop_eye(aligned_img[0], aligned_shapes_cur[0])
            self.left_eyes.append(leye)
            self.right_eyes.append(reye)
            im_resized = cv2.resize(im, None, None, fx=factor, fy=factor)
            self.resized_imgs.append(im_resized)

        # For visualize
        self.plot_vis_list = []
        self.total_eye1_prob = []
        self.total_eye2_prob = []

    def _set_up_dlib(self):
        # Note that CUDA dir should be /usr/local/cuda and without AVX intructions
        pwd = os.path.dirname(os.path.abspath(__file__))
        # self.cnn_face_detector = dlib.cnn_face_detection_model_v1(pwd + '/mmod_human_face_detector.dat')
        self.front_face_detector = dlib.get_frontal_face_detector()
        self.lmark_predictor = dlib.shape_predictor(pwd + '/dlib_model/shape_predictor_68_face_landmarks.dat')

    def gen_videos(self, out_dir, tag=''):
        vid_name = os.path.basename(self.input_vid_path)
        out_path = os.path.join(out_dir, tag + '_' + vid_name)
        print('Generating video: {}'.format(out_path))
        # Output folder
        if not os.path.exists(os.path.dirname(out_dir)):
            os.makedirs(os.path.dirname(out_dir))

        final_list = []
        for i in tqdm(range(self.frame_num)):
            final_vis = np.concatenate([self.resized_imgs[i], self.plot_vis_list[i]], axis=1)
            final_list.append(final_vis)
        pv.gen_vid(out_path, final_list, self.fps)

    def get_eye_by_fid(self, i):
        eye1, eye2 = self.left_eyes[i], self.right_eyes[i]
        return eye1, eye2

    def push_eye_prob(self, eye1_prob, eye2_prob):
        self.total_eye1_prob.append(eye1_prob)
        self.total_eye2_prob.append(eye2_prob)

    def plot_by_fid(self, i):
        # Vis plots
        max_X = self.frame_num / self.fps
        params = {}
        params['title'] = 'Eye-state-probability'
        params['colors'] = ['b-']
        params['markers'] = [None]
        params['linewidth'] = 3
        params['markersize'] = None
        params['figsize'] = None

        x_axis = np.arange(self.frame_num) / self.fps
        # Vis plots
        prob_plot_1 = plot.draw2D([x_axis[:i + 1]],
                                [self.total_eye1_prob],
                                order=[''],
                                xname='time',
                                yname='eye state',
                                params=params,
                                xlim=[0, max_X],
                                ylim=[-1, 2])

        prob_plot_2 = plot.draw2D([x_axis[:i + 1]],
                                [self.total_eye2_prob],
                                order=[''],
                                xname='time',
                                yname='eye state',
                                params=params,
                                xlim=[0, max_X],
                                ylim=[-1, 2])

        vis = np.concatenate([prob_plot_1, prob_plot_2], axis=1)
        scale = float(self.output_height) / vis.shape[0]
        # Resize plot size to same size with video
        vis = cv2.resize(vis, None, None, fx=scale, fy=scale)
        self.plot_vis_list.append(vis)
        return self.plot_vis_list

In [ ]:
import tensorflow as tf
import os, cv2
from deep_base.ops import get_restore_var_list
import yaml, os
from easydict import EasyDict as edict
pwd = os.path.dirname(__file__)


class Solver(object):
    """
    Solver for training and testing
    """
    def __init__(self,
                 sess,
                 net,
                 mode='cnn'):
        cfg_file = os.path.join(pwd, 'blink_{}.yml'.format(mode))
        with open(cfg_file, 'r') as f:
            cfg = edict(yaml.load(f))

        self.sess = sess
        self.net = net
        self.cfg = cfg
        self.mode = mode

    def init(self):
        cfg = self.cfg
        self.img_size = cfg.IMG_SIZE
        pwd = os.path.dirname(os.path.abspath(__file__))
        self.summary_dir = os.path.join(pwd, cfg.SUMMARY_DIR)
        if not os.path.exists(self.summary_dir):
            os.makedirs(self.summary_dir)

        self.model_dir = os.path.join(pwd, cfg.MODEL_DIR)
        if not os.path.exists(self.model_dir):
            os.makedirs(self.model_dir)
        self.model_path = os.path.join(self.model_dir, 'model.ckpt')
        self.global_step = tf.Variable(0, trainable=False, name='global_step')
        self.saver = tf.train.Saver(max_to_keep=5)
        # initialize the graph
        if self.net.is_train:
            self.num_epoch = cfg.TRAIN.NUM_EPOCH
            self.learning_rate = cfg.TRAIN.LEARNING_RATE
            self.decay_rate = cfg.TRAIN.DECAY_RATE
            self.decay_step = cfg.TRAIN.DECAY_STEP
            self.net.loss()
            self.set_optimizer()
            # Add summary
            self.loss_summary = tf.summary.scalar('loss_summary', self.net.total_loss)
            self.lr_summary = tf.summary.scalar('learning_rate_summary', self.LR)
            self.summary = tf.summary.merge([self.loss_summary, self.lr_summary])
            self.writer = tf.summary.FileWriter(self.summary_dir, self.sess.graph)

        self.sess.run(tf.global_variables_initializer())
        self.load()

    def train(self, *args):
        if self.mode == 'cnn':
            return self.train_cnn(images=args[0], labels=args[1])
        elif self.mode == 'lrcn':
            return self.train_lrcn(seq_tensor=args[0],
                                   len_list=args[1],
                                   state_list=args[2])
        else:
            raise ValueError('We only support mode = [cnn, lrcn]...')

    def test(self, *args):
        if self.mode == 'cnn':
            return self.test_cnn(images=args[0])
        elif self.mode == 'lrcn':
            return self.test_lrcn(seq_tensor=args[0],
                                   len_list=args[1])
        else:
            raise ValueError('We only support mode = [cnn, lrcn]...')

    def test_cnn(self, images):
        # Check input size
        for i, im in enumerate(images):
            images[i] = cv2.resize(im, (self.img_size[0], self.img_size[1]))

        feed_dict = {
            self.net.input: images,
        }
        fetch_list = [
            self.net.prob,
        ]
        return self.sess.run(fetch_list, feed_dict=feed_dict)

    def train_cnn(self, images, labels):
        feed_dict = {
            self.net.input: images,
            self.net.gt: labels
        }
        fetch_list = [
            self.train_op,
            self.summary,
            self.net.prob,
            self.net.net_loss,
        ]
        return self.sess.run(fetch_list, feed_dict=feed_dict)

    def test_lrcn(self, seq_tensor, len_list):
        feed_dict = {
            self.net.input: seq_tensor,
            self.net.seq_len: len_list,
        }
        fetch_list = [
            self.net.prob,
        ]
        return self.sess.run(fetch_list, feed_dict=feed_dict)

    def train_lrcn(self, seq_tensor, len_list, state_list):
        feed_dict = {
            self.net.input: seq_tensor,
            self.net.seq_len: len_list,
            self.net.eye_state_gt: state_list
        }
        fetch_list = [
            self.train_op,
            self.summary,
            self.net.prob,
            self.net.net_loss,

        ]
        return self.sess.run(fetch_list, feed_dict=feed_dict)

    def save(self, step):
        """ Save checkpoints """
        save_path = self.saver.save(self.sess, self.model_path, global_step=step)
        print('Model {} saved in file.'.format(save_path))

    def load(self):
        """Load weights from checkpoint"""
        if os.path.isfile(self.model_path + '.meta'):
            variables_to_restore = get_restore_var_list(self.model_path)
            restorer = tf.train.Saver(variables_to_restore)
            restorer.restore(self.sess, self.model_path)
            print('Loading checkpoint {}'.format(self.model_path))
        else:
            print('Loading failed.')

    def set_optimizer(self):
        # Set learning rate decay
        self.LR = tf.train.exponential_decay(
            learning_rate=self.learning_rate,
            global_step=self.global_step,
            decay_steps=self.decay_step,
            decay_rate=self.decay_rate,
            staircase=True
        )
        if self.cfg.TRAIN.METHOD == 'SGD':
            optimizer = tf.train.GradientDescentOptimizer(
                learning_rate=self.LR,
            )
        elif self.cfg.TRAIN.METHOD == 'Adam':
            optimizer = tf.train.AdamOptimizer(
                learning_rate=self.LR,
            )
        else:
            raise ValueError('We only support [SGD, Adam] right now...')

        self.train_op = optimizer.minimize(
            loss=self.net.total_loss,
            global_step=self.global_step,
            var_list=None)

In [ ]:
# import config as cfg
import dlib
import cv2
import numpy as np
from imutils import face_utils
from scipy.spatial import distance as dist



class eye_blink_detector():
    def __init__(self):
        # cargar modelo para deteccion de puntos de ojos
        self.predictor_eyes = dlib.shape_predictor(cfg.eye_landmarks)

    def eye_blink(self,gray,rect,COUNTER,TOTAL):
        (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
        (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = self.predictor_eyes(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = self.eye_aspect_ratio(leftEye)
        rightEAR = self.eye_aspect_ratio(rightEye)
        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0
        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if ear < cfg.EYE_AR_THRESH:
            COUNTER += 1
        # otherwise, the eye aspect ratio is not below the blink
        # threshold
        else:
            # if the eyes were closed for a sufficient number of
            # then increment the total number of blinks
            if COUNTER >= cfg.EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
            # reset the eye frame counter
            COUNTER = 0
        return COUNTER,TOTAL

    def eye_aspect_ratio(self,eye):
        # compute the euclidean distances between the two sets of
        # vertical eye landmarks (x, y)-coordinates
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        # compute the euclidean distance between the horizontal
        # eye landmark (x, y)-coordinates
        C = dist.euclidean(eye[0], eye[3])
        # compute the eye aspect ratio
        ear = (A + B) / (2.0 * C)
        # return the eye aspect ratio
        return ear


In [ ]:
import tensorflow as tf
import os, cv2
from deep_base.ops import get_restore_var_list
import yaml, os
from easydict import EasyDict as edict
pwd = os.path.dirname(__file__)


class Solver(object):
    """
    Solver for training and testing
    """
    def __init__(self,
                 sess,
                 net,
                 mode='cnn'):
        cfg_file = os.path.join(pwd, 'blink_{}.yml'.format(mode))
        with open(cfg_file, 'r') as f:
            cfg = edict(yaml.load(f))

        self.sess = sess
        self.net = net
        self.cfg = cfg
        self.mode = mode

    def init(self):
        cfg = self.cfg
        self.img_size = cfg.IMG_SIZE
        pwd = os.path.dirname(os.path.abspath(__file__))
        self.summary_dir = os.path.join(pwd, cfg.SUMMARY_DIR)
        if not os.path.exists(self.summary_dir):
            os.makedirs(self.summary_dir)

        self.model_dir = os.path.join(pwd, cfg.MODEL_DIR)
        if not os.path.exists(self.model_dir):
            os.makedirs(self.model_dir)
        self.model_path = os.path.join(self.model_dir, 'model.ckpt')
        self.global_step = tf.Variable(0, trainable=False, name='global_step')
        self.saver = tf.train.Saver(max_to_keep=5)
        # initialize the graph
        if self.net.is_train:
            self.num_epoch = cfg.TRAIN.NUM_EPOCH
            self.learning_rate = cfg.TRAIN.LEARNING_RATE
            self.decay_rate = cfg.TRAIN.DECAY_RATE
            self.decay_step = cfg.TRAIN.DECAY_STEP
            self.net.loss()
            self.set_optimizer()
            # Add summary
            self.loss_summary = tf.summary.scalar('loss_summary', self.net.total_loss)
            self.lr_summary = tf.summary.scalar('learning_rate_summary', self.LR)
            self.summary = tf.summary.merge([self.loss_summary, self.lr_summary])
            self.writer = tf.summary.FileWriter(self.summary_dir, self.sess.graph)

        self.sess.run(tf.global_variables_initializer())
        self.load()

    def train(self, *args):
        if self.mode == 'cnn':
            return self.train_cnn(images=args[0], labels=args[1])
        elif self.mode == 'lrcn':
            return self.train_lrcn(seq_tensor=args[0],
                                   len_list=args[1],
                                   state_list=args[2])
        else:
            raise ValueError('We only support mode = [cnn, lrcn]...')

    def test(self, *args):
        if self.mode == 'cnn':
            return self.test_cnn(images=args[0])
        elif self.mode == 'lrcn':
            return self.test_lrcn(seq_tensor=args[0],
                                   len_list=args[1])
        else:
            raise ValueError('We only support mode = [cnn, lrcn]...')

    def test_cnn(self, images):
        # Check input size
        for i, im in enumerate(images):
            images[i] = cv2.resize(im, (self.img_size[0], self.img_size[1]))

        feed_dict = {
            self.net.input: images,
        }
        fetch_list = [
            self.net.prob,
        ]
        return self.sess.run(fetch_list, feed_dict=feed_dict)

    def train_cnn(self, images, labels):
        feed_dict = {
            self.net.input: images,
            self.net.gt: labels
        }
        fetch_list = [
            self.train_op,
            self.summary,
            self.net.prob,
            self.net.net_loss,
        ]
        return self.sess.run(fetch_list, feed_dict=feed_dict)

    def test_lrcn(self, seq_tensor, len_list):
        feed_dict = {
            self.net.input: seq_tensor,
            self.net.seq_len: len_list,
        }
        fetch_list = [
            self.net.prob,
        ]
        return self.sess.run(fetch_list, feed_dict=feed_dict)

    def train_lrcn(self, seq_tensor, len_list, state_list):
        feed_dict = {
            self.net.input: seq_tensor,
            self.net.seq_len: len_list,
            self.net.eye_state_gt: state_list
        }
        fetch_list = [
            self.train_op,
            self.summary,
            self.net.prob,
            self.net.net_loss,

        ]
        return self.sess.run(fetch_list, feed_dict=feed_dict)

    def save(self, step):
        """ Save checkpoints """
        save_path = self.saver.save(self.sess, self.model_path, global_step=step)
        print('Model {} saved in file.'.format(save_path))

    def load(self):
        """Load weights from checkpoint"""
        if os.path.isfile(self.model_path + '.meta'):
            variables_to_restore = get_restore_var_list(self.model_path)
            restorer = tf.train.Saver(variables_to_restore)
            restorer.restore(self.sess, self.model_path)
            print('Loading checkpoint {}'.format(self.model_path))
        else:
            print('Loading failed.')

    def set_optimizer(self):
        # Set learning rate decay
        self.LR = tf.train.exponential_decay(
            learning_rate=self.learning_rate,
            global_step=self.global_step,
            decay_steps=self.decay_step,
            decay_rate=self.decay_rate,
            staircase=True
        )
        if self.cfg.TRAIN.METHOD == 'SGD':
            optimizer = tf.train.GradientDescentOptimizer(
                learning_rate=self.LR,
            )
        elif self.cfg.TRAIN.METHOD == 'Adam':
            optimizer = tf.train.AdamOptimizer(
                learning_rate=self.LR,
            )
        else:
            raise ValueError('We only support [SGD, Adam] right now...')

        self.train_op = optimizer.minimize(
            loss=self.net.total_loss,
            global_step=self.global_step,
            var_list=None)


In [ ]:
#Visualize Video result with Labels
import imutils
from imutils.video import FileVideoStream, VideoStream
vs = VideoStream(src=0).start()

frame = imutils.resize(frame, width=450)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
rects = eye_blink_detector(gray, 0) #Key Model
for rect in rects:
		shape = Solver(gray, rect)
		shape = face_utils.shape_to_np(shape)

		# extract the left and right eye coordinates, then use the
		leftEye = shape[lStart:lEnd]
		rightEye = shape[rStart:rEnd]
		# visualize each of the eyes
		leftEyeHull = cv2.convexHull(leftEye)
		rightEyeHull = cv2.convexHull(rightEye)
		cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
		cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
		# check to see if the eye aspect ratio is below the blink
		# threshold, and if so, increment the blink frame counter
		if ear < EYE_AR_THRESH:
			COUNTER += 1
    else
      COUNTER = 0

cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)'

cv2.imshow("Frame", frame)

cv2.destroyAllWindows()
vs.stop()

In [ ]:
#Blink Frequency Graph
slopes.append((eye_heights[i]-eye_heights[i+1]))
avg_slope = np.sum(slopes)/len(slopes) #average slope
min_slope = min(slopes) #minimum slope

print(abs(avg_slope-min_slope))

plt.axline(xy1=(0,avg_slope),slope=0)
plt.axline(xy1=(0,min_slope),slope=0)
plt.plot(slopes)
plt.plot(eye_heights)
plt.scatter(x_axis, eye_heights)
plt.grid()
plt.show()

In [ ]:
! jupyter nbconvert --to html /content/Inoculi_liveliness1_0ipynb.ipynb

[NbConvertApp] Converting notebook /content/Inoculi_liveliness1_0ipynb.ipynb to html
[NbConvertApp] Writing 712165 bytes to /content/Inoculi_liveliness1_0ipynb.html
